In [1]:
import toy_data.models as m
import toy_data.visualize as vs
import toy_data.data_types as tp
import numpy as np
import bokeh.io
import bokeh.plotting as bp
import bokeh.charts as bc
import tensorflow as tf
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
t0 = 0
interv = 0.1
n_l = 200 
t_l = t0 + n_l * interv 
n_c = 200
t_c = t0 + (n_c + n_l) * interv
t = np.linspace(t0, t_l, n_l)
data_set = tp.DataSet.from_X(t[:, np.newaxis], m.Sine_1D(A=1))
vs.regression_1D(data_set, fig_height=300, fig_width=900)

In [3]:
data_set.y.shape
d_x = data_set.y

In [4]:
d_X = d_x[np.newaxis, :]
x_dim = 1
d_X.shape # No. of samples, Time, No. of dimensions

(1, 200, 1)

In [5]:
batch_size = 1

In [6]:
ef = lambda x: tf.expand_dims(x, -1) # expand forward
eb = lambda x: tf.expand_dims(x, 0) # expand backward

In [30]:
state_size = 10
tf.reset_default_graph()
x_in = tf.placeholder(tf.float32, [batch_size, None, x_dim]) # [batch_size, num_steps]
x_out = tf.placeholder(tf.float32, [batch_size, None, x_dim]) # [batch_size, num_steps]
cell = tf.contrib.rnn.BasicLSTMCell(state_size)

init_h = tf.get_variable('init_h', (1, state_size), initializer=tf.truncated_normal_initializer())
init_c = tf.get_variable('init_c', (1, state_size), initializer=tf.truncated_normal_initializer())
init_state_tuple = tf.nn.rnn_cell.LSTMStateTuple(init_c, init_h)

rnn_out, final_state = tf.nn.dynamic_rnn(cell, x_in, dtype=tf.float32, initial_state=init_state_tuple)

W = tf.get_variable('output_embedding', (state_size, x_dim), initializer=tf.truncated_normal_initializer())
out_pred = tf.reduce_sum(ef(rnn_out) * eb(eb(W)), axis=-2)

loss_all = tf.square(out_pred - x_out)
loss = tf.reduce_sum(loss_all)
train_step = tf.train.AdamOptimizer().minimize(loss)

In [31]:
# d_x_noisy = data_set.y + np.random.normal(scale=0.5, size=data_set.y.shape)
# d_X_noisy = d_x_noisy[np.newaxis, :]
inputs = {x_in: d_X[:, :-1, :], x_out: d_X[:, 1:, :]}

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [32]:
for i in range(1000):
    sess.run(train_step, feed_dict=inputs)
print(sess.run(loss, feed_dict=inputs))

0.00837597


In [33]:
val_actual = sess.run(x_out, feed_dict=inputs).ravel()
val_pred = sess.run(out_pred, feed_dict=inputs).ravel()

In [34]:
f = bp.figure(plot_width=900, plot_height=300)
n_prev = n_l - 1
cvert = lambda a, b: list(np.array([a, b]).T)
f.multi_line(cvert(t[-n_prev:], t[-n_prev:]),
             cvert(val_actual[-n_prev:], val_pred[-n_prev:]),
             line_width=4, color='purple', legend='error')
f.circle(t[-n_prev:], val_actual[-n_prev:], color='black', legend='original')
f.circle(t[-n_prev:], val_pred[-n_prev:], legend='predicted')
bp.show(f)

## Predict beyond length of training samples

In [35]:
gen_seq_len = tf.placeholder(tf.int32, [])
gen_seq_states = tf.transpose(tf.tile(final_state, (1, gen_seq_len, 1)), [1, 0, 2])

def update_state(_state, _):
    h = eb(_state[1, :])
    c = eb(_state[0, :])
    _pred = tf.matmul(h, W)
    _state_tuple = tf.nn.rnn_cell.LSTMStateTuple(c, h)
    _, _next_state = cell(_pred, _state_tuple)
    return tf.concat((_next_state.c, _next_state.h), axis=0)
    
next_states = tf.scan(update_state, gen_seq_states)
next_preds = tf.map_fn(lambda s: tf.matmul(eb(s[1, :]), W), next_states)


In [36]:
gen_inputs = lambda n: {x_in: d_X[:, :-1, :], gen_seq_len: n}

val_continue = sess.run(next_preds, feed_dict=gen_inputs(n_c)).ravel()
f = bp.figure(plot_width=700, plot_height=300)
n_total = n_l + n_c
f.circle(np.arange(n_l - n_prev, n_total), np.append(val_pred[-n_prev:], val_continue), color='red', legend='prediction')
f.circle(np.arange(n_l - n_prev, n_l), val_actual[-n_prev:], legend='orignal')
bp.show(f)